# Learning Process-consistent Knowledge Tracing(LPKT)

This notebook will show you how to train and use the LPKT.
First, we will show how to get the data (here we use assistment-2017 as the dataset).
Then we will show how to train a LPKT and perform the parameters persistence.
At last, we will show how to load the parameters from the file and evaluate on the test dataset.

The script version could be found in [LPKT.py](LPKT.ipynb)


## Data Preparation

Before we process the data, we need to first acquire the dataset which is shown in [prepare_dataset.ipynb](prepare_dataset.ipynb)

In [2]:
import numpy as np
from load_data import DATA

def generate_q_matrix(path, n_skill, n_problem, gamma=0):
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            problem2skill = eval(line)
    q_matrix = np.zeros((n_problem + 1, n_skill + 1)) + gamma
    for p in problem2skill.keys():
        q_matrix[p][problem2skill[p]] = 1
    return q_matrix

batch_size = 32
n_at = 1326
n_it = 2839
n_question = 102
n_exercise = 3162
seqlen = 500
d_k = 128
d_a = 50
d_e = 128
q_gamma = 0.03
dropout = 0.2

dat = DATA(seqlen=seqlen, separate_char=',')
train_data = dat.load_data('../../data/anonymized_full_release_competition_dataset/train0.txt')
test_data = dat.load_data('../../data/anonymized_full_release_competition_dataset/test.txt')
q_matrix = generate_q_matrix(
    '../../data/anonymized_full_release_competition_dataset/problem2skill',
    n_question, n_exercise,
    q_gamma
)

## Training and Persistence

In [3]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [4]:
from EduKTM import LPKT

lpkt = LPKT(n_at, n_it, n_exercise, n_question, d_a, d_e, d_k, q_matrix, batch_size, dropout)
lpkt.train(train_data, test_data, epoch=2, lr=0.003)
lpkt.save("lpkt.params")

Training: 100%|██████████| 240/240 [07:46<00:00,  1.94s/it]


MemoryError: Unable to allocate 803. MiB for an array with shape (499, 32, 103, 128) and data type float32

## Loading and Testing

In [4]:
lpkt.load("lpkt.params")
_, auc, accuracy = lpkt.eval(test_data)
print("auc: %.6f, accuracy: %.6f" % (auc, accuracy))

INFO:root:load parameters from lpkt.params
Testing: 100%|██████████| 26/26 [01:18<00:00,  3.04s/it]

auc: 0.717543, accuracy: 0.682427
